# Deep Convolutional Neural Network (DCGAN) - Implementation

The purpose of this notebook is to implement the Deep Convolutional Neural Network architecture, as outlined in section 3.4.3 of the bachelor thesis.

The code provided in this notebook was developed using the Google Colab platform.

The code in this notebook incorporates the following sources as references:

- https://pytorch.org/tutorials/beginner/dcgan_faces_tutorial.html

## Step 1 - Importing Dependencies

- Importing the necessary libraries to execute the code.

In [1]:
import os
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
import torchvision.transforms as transforms
import torchvision.utils as vutils
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML
from torch.utils.data import Dataset, DataLoader
import pandas as pd
from PIL import Image
import seaborn as sns

## Step 2 - Hyperparameter Settings

- Set the HPs for the DCGAN deep generative model. Besides, also check whether a GPU is available for use.

In [ ]:
torch.manual_seed(999)  # Manual definiton of the seed
workers = 2             # Number of workers for dataloader
batch_size = 8          # Batch size during training
image_size = 64         # Spatial size of training images. All images will be resized to this size using a transformer.
nc = 3                  # Number of channels in the training images. For color images, this is 3
nz = 100                # Size of z latent vector (i.e., size of generator input)
ngf = 64                # Size of feature maps in the generator
ndf = 64                # Size of feature maps in the discriminator
num_epochs = 800        # Number of training epochs
lr = 0.0002             # Learning rate for optimizers
beta1 = 0.5             # Beta1 hyperparam for Adam optimizers
ngpu = 1                # Number of GPUs available. Use 0 for CPU mode.

device = torch.device("cuda:0" if (torch.cuda.is_available() and ngpu > 0) else "cpu")
print(f'Selected device: {device}')

- Function to initializing the weights of the DCGAN model as definined in the original whitepaper.

In [ ]:
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        nn.init.normal_(m.weight.data, 1.0, 0.02)
        nn.init.constant_(m.bias.data, 0)

## Step 3 - Dataset Loading

- Defining the custom class for loading the images as PyTorch dataset.

In [4]:
class Dataset(Dataset):
    def __init__(self, labels_file, root_dir, transform=None):
        self.annotations = pd.read_csv(labels_file, header=None)
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.annotations)

    def __getitem__(self, index):
        img_path = os.path.join(self.root_dir, str(self.annotations.iloc[index, 1]), self.annotations.iloc[index, 0])
        image = Image.open(img_path)
        image = image.convert("RGB")
        label = torch.tensor(int(self.annotations.iloc[(index, 2)]))

        if self.transform:
            image = self.transform(image)

        return(image, label)
    
    def __getlabel__(self, index):
        label = (self.annotations.iloc[(index, 1)])        

        return(label)

- The preprocessing transformation matchs the data with the expected format from the DCGAN model.
- The labels .cvs file should be passed in a class-wise definiton, since the model is unconditional will generate one class per time.

In [ ]:
preprocessing = transforms.Compose([transforms.Resize(image_size), 
                                    transforms.ToTensor(),
                                    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
                                   ])

labels_file = '/path/to/class/labels/csv'
root_dir = '/path/to/root/image/folder'

dataset = Dataset(labels_file=labels_file, root_dir=root_dir, transform=preprocessing)
dataloader = DataLoader(dataset=dataset, batch_size=batch_size, shuffle=True, num_workers=workers)

## Step 4 - DCGAN Model Definition

- **Generator:** Defining the Generator of the DCGAN model, the Generator has the role to create synthetic instances from a noise input.

In [11]:
class Generator(nn.Module):
    def __init__(self, ngpu):
        super(Generator, self).__init__()
        self.ngpu = ngpu
        self.main = nn.Sequential(
            nn.ConvTranspose2d( nz, ngf * 16, 4, 1, 0, bias=False),
            nn.BatchNorm2d(ngf * 16),
            nn.ReLU(True),
            nn.ConvTranspose2d( ngf * 16, ngf * 8, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 8),
            nn.ReLU(True),
            nn.ConvTranspose2d(ngf * 8, ngf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 4),
            nn.ReLU(True),
            nn.ConvTranspose2d( ngf * 4, ngf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 2),
            nn.ReLU(True),
            nn.ConvTranspose2d( ngf * 2, nc, 4, 2, 1, bias=False),
            nn.Tanh()
        )

    def forward(self, input):
        return self.main(input)

In [ ]:
netG = Generator(ngpu).to(device)

if (device.type == 'cuda') and (ngpu > 1):
    netG = nn.DataParallel(netG, list(range(ngpu)))

netG.apply(weights_init)

print(netG)

- **Discriminator:** Defining the Discriminator of the DCGAN model, the Discriminator has the role of predict whether an image is fake or real.

In [13]:
class Discriminator(nn.Module):
    def __init__(self, ngpu):
        super(Discriminator, self).__init__()
        self.ngpu = ngpu
        self.main = nn.Sequential(
            nn.Conv2d(nc, ndf, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(ndf, ndf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 2),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(ndf * 2, ndf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 4),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(ndf * 4, ndf * 8, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 8),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(ndf * 8, 1, 4, 1, 0, bias=False),
            nn.Sigmoid()
        )

    def forward(self, input):
        return self.main(input)

In [ ]:
netD = Discriminator(ngpu).to(device)

if (device.type == 'cuda') and (ngpu > 1):
    netD = nn.DataParallel(netD, list(range(ngpu)))

netD.apply(weights_init)

print(netD)

- Defining the loss function and optimizers for the DCGAN model.
- Setting the fixed noise to evaluated the evolution of the DCGAN model over time when training.

In [ ]:
criterion = nn.BCELoss()

optimizerD = optim.Adam(netD.parameters(), lr=lr, betas=(beta1, 0.999))
optimizerG = optim.Adam(netG.parameters(), lr=lr, betas=(beta1, 0.999))

fixed_noise = torch.randn(224, nz, 1, 1, device=device)
real_label = 1.
fake_label = 0.

## Step 5 - Training the DCGAN

- Defining the training loop for the DCGAN, the code also tracks values of interest for future visualization.

In [ ]:
img_list = []
G_losses = []
D_losses = []
G_losses_epoch = []
D_losses_epoch = []
iters = 0

print("Starting Training Loop...")
for epoch in range(num_epochs):
    for i, data in enumerate(dataloader, 0):

        # (1) Update D network

        netD.zero_grad()

        real_cpu = data[0].to(device)
        b_size = real_cpu.size(0)
        label = torch.full((b_size,), real_label, dtype=torch.float, device=device)
        output = netD(real_cpu).view(-1)

        errD_real = criterion(output, label)
        errD_real.backward()
        D_x = output.mean().item()

        noise = torch.randn(b_size, nz, 1, 1, device=device)

        fake = netG(noise)
        label.fill_(fake_label)
        output = netD(fake.detach()).view(-1)

        errD_fake = criterion(output, label)
        errD_fake.backward()
        D_G_z1 = output.mean().item()
        errD = errD_real + errD_fake

        optimizerD.step()

        # (2) Update G network

        netG.zero_grad()
        label.fill_(real_label)
        output = netD(fake).view(-1)

        errG = criterion(output, label)
        errG.backward()
        D_G_z2 = output.mean().item()

        optimizerG.step()

        if i % 5 == 0:
            print('[%d/%d][%d/%d]\tLoss_D: %.4f\tLoss_G: %.4f\tD(x): %.4f\tD(G(z)): %.4f / %.4f'
                  % (epoch, num_epochs, i, len(dataloader),
                     errD.item(), errG.item(), D_x, D_G_z1, D_G_z2))

        G_losses.append(errG.item())
        D_losses.append(errD.item())

        if (iters % 1000 == 0) or ((epoch == num_epochs-1) and (i == len(dataloader)-1)):
            with torch.no_grad():
                fake = netG(fixed_noise).detach().cpu()
            img_list.append(vutils.make_grid(fake, padding=2, normalize=True))

        iters += 1

    G_losses_epoch.append(sum(G_losses)/len(G_losses))
    D_losses_epoch.append(sum(D_losses)/len(D_losses))

## Step 6 - Visualizing the Results

- Visualizing the evolution of the synthetic data created by DCGAN model over epochs.

In [ ]:
fig = plt.figure(figsize=(8,8))
plt.axis("off")
ims = [[plt.imshow(np.transpose(i,(1,2,0)), animated=True)] for i in img_list]
ani = animation.ArtistAnimation(fig, ims, interval=1000, repeat_delay=1000, blit=True)

HTML(ani.to_jshtml())

- Visualizing the Generator and Discriminator loss over epochs.

In [ ]:
sns.set_theme(style="whitegrid")
plt.figure(figsize=(10,5))
plt.title("Generator and discriminator training loss")
plt.plot(G_losses_epoch,label="G")
plt.plot(D_losses_epoch,label="D")
plt.xlabel("epochs")
plt.ylabel("Loss")
plt.legend()
plt.show()

## Step 7 - Saving the Synthetic Images

- Defining a fucntion to create a new images based on a noise input.
- Saving images in a desired folder location.
- Defining the desired number of images in total after the synthetic augmentation
- As mentioned, the images are created per class since this is a unconditional implementation.

In [ ]:
def save_synthetic_data(dataset, nz, save_path, num_instances):
  num_images = num_instances - dataset.__len__()
  noise = torch.randn(num_images, nz, 1, 1, device=device)
  image_tensor  = netG(noise)
  image_list = []
  for i in range(image_tensor.size(0)):
    tensor_image = image_tensor[i].detach().cpu()
    tensor_image = (tensor_image*0.5) + 0.5
    pil_image = transforms.ToPILImage()(tensor_image)
    pil_image = transforms.Resize((224, 224))(pil_image)
    pil_image.save(os.path.join(save_path, dataset.__getlabel__(0), 'dcgan_'+str(dataset.__getlabel__(0))+'_'+str(i)+'.jpg'))


In [ ]:
save_path = "/path/to/save/images/folder"
num_instances = 1000

save_synthetic_data(dataset, nz, save_path, num_instances)